# Analyze Fundamental Stock Data

In [4]:
!pip install eod
# Libraries
import pandas as pd
from eod import EodHistoricalData
from functools import reduce
from datetime import datetime, timedelta

In [ ]:
# Importing and assigning the api key
with open("../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()

In [5]:
    
# EOD Historical Data client
client = EodHistoricalData("6190a6350b56a7.50134497")

# Formatting Fundamental Data

In [7]:
def getFundamentals(ticker):
    """
    Returns the fundamental data from the financial data API.  Combines the quarterly balance 
    sheet, cash flow, income statement, and earnings for a specific stock ticker.
    """
    
    # Getting data
    fund_data = client.get_fundamental_equity(ticker)
    
    # Financials
    bal = pd.DataFrame(fund_data['Financials']['Balance_Sheet']['quarterly']).T
    
    cf = pd.DataFrame(fund_data['Financials']['Cash_Flow']['quarterly']).T
    
    inc = pd.DataFrame(fund_data['Financials']['Income_Statement']['quarterly']).T
    
    # Earnings
    earn = pd.DataFrame(fund_data['Earnings']['History']).T
    
    # Merging them together
    df = reduce(
        lambda left,right: pd.merge(
            left,
            right,
            left_index=True, 
            right_index=True, 
            how='outer',
            suffixes=('', '_drop')
        ), 
        [bal, cf, inc, earn]
    )
    
    # Dropping redundant date and duplicate columns
    dup_cols = [i for i in df.columns if "date" in i or "Date" in i or "_drop" in i]
    
    df = df.drop(dup_cols, axis=1)
    
    return df

In [8]:
def getPrices(df, ticker):
    """
    Gets the stock price at the time for each date in the financial statements for
    the given ticker and dataframe of financial information.
    """
    # Getting stock price at the time
    prices = client.get_prices_eod(ticker, period='d')
    
    prices = pd.DataFrame(prices).set_index('date')[['adjusted_close', 'close', 'volume']]

    # Converting to date time
    prices.index = pd.to_datetime(prices.index)

    # Filling in missing price data
    prices = prices.reindex(
        pd.date_range(prices.index[0], prices.index[-1]),
        method='ffill'
    )
    
    # Converting back to string for merging later
    prices.index = prices.index.strftime("%Y-%m-%d")
        
    price_dates = [i for i in prices.index if i in df.index]
    
    prices = prices.loc[price_dates]

    # Joining together
    df = df.join(prices, how='outer')
    
    return df

In [9]:
def formatFundamentals(ticker, dropna=False):
    """
    Formats the given ticker's fundamental and price data. Cleans up the data by dropping
    any empty/nan values if requested.
    """
    
    # Getting fundamental data
    fund_data = getFundamentals(ticker)
    
    # Getting accompanying price data
    df = getPrices(fund_data, ticker)
    
    # Dropping if all items are na in respective row
    df = df.dropna(how='all')
    
    if dropna:
        # Dropping mostly nan columns and rows if requested
        df = df.dropna(
            axis=0,
            thresh=round(df.shape[0]*.3) # If 50% of the values in the row are Nans, drop the whole row
        ).dropna(
            axis=1,
            thresh=round(df.shape[1]*.3) # If 50% of the values in the columns are Nans, drop the whole column
        )
    
    return df

In [10]:
df = formatFundamentals("TSLA", dropna=True)

HTTPError: 403 Client Error: Forbidden for url: https://eodhistoricaldata.com/api/fundamentals/TSLA?fmt=json&api_token=6190a6350b56a7.50134497

In [ ]:
df

# Getting Fundamentals from Multiple Companies

In [ ]:
def getMultipleFunds(tickers, api_token):
    """
    Gets fundamental data from multiple stock tickers given as a list. Returns
    a large dataframe containing the concatenated information for all the given
    tickers.
    """
    
    # Verifying if the list of tickers is compatible
    available = client.get_exchange_symbols("US")

    available = set(i['Code'] for i in available)
    
    tickers = [i for i in tickers if i in available]
    
    if len(tickers)==0:
        return "No valid tickers found."
    
    # Iterating through the tickers
    dfs = {}
    
    for ticker in tickers:
        
        dfs[ticker] = formatFundamentals(ticker)
        
    
    return pd.concat(dfs, axis=0)
    

In [ ]:
df = getMultipleFunds(["asdfase"], api_key)

In [ ]:
df#.loc['TSLA']